In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
data_chemicals = pd.read_csv("chemicals.csv")
data_drought = pd.read_csv("droughts.csv")
data_education = pd.read_csv("education_attainment.csv", encoding = 'latin1')
data_water = pd.read_csv("water_usage.csv")
water_info = pd.read_csv("water_usage_dictionary.csv")

In [6]:
data_chemicals.head()

,cws_name,chemical_species,contaminant_level,county,pws_id,pop_served,state,unit_measurement,value,year,fips,state_fips
0,CALIFORNIA WATER SERVICE - LIVERMORE,Uranium,Less than or equal MCL,Alameda,CA0110003,57200,California,micrograms/L,1.54,2000,6001,6
1,CITY OF LIVERMORE,Uranium,Less than or equal MCL,Alameda,CA0110011,26400,California,micrograms/L,1.64,2000,6001,6
2,CITY OF PLEASANTON,Uranium,Less than or equal MCL,Alameda,CA0110008,70600,California,micrograms/L,1.64,2000,6001,6
3,DUBLIN SAN RAMON SERVICES DISTRICT,Uranium,Less than or equal MCL,Alameda,CA0110009,72500,California,micrograms/L,1.64,2000,6001,6
4,NORRIS CANYON PROPERTY OWNERS ASSN,Uranium,Less than or equal MCL,Alameda,CA0103040,100,California,micrograms/L,4.51,2000,6001,6


## Pollution index by county

In [7]:
data_chemicals['greater_than_MCL'] = (data_chemicals['contaminant_level'] == "Greater than MCL") * 1.
groups = data_chemicals.groupby("cws_name")[['greater_than_MCL', 'fips', "pop_served"]].mean()
groups['index_with_weights'] = groups['greater_than_MCL']*groups['pop_served']
chemicals_index = groups.groupby("fips")['index_with_weights'].sum() / groups.groupby("fips")['pop_served'].sum()

In [8]:
chemicals_index

fips
6001.000000     0.000000
6003.000000     0.000000
6005.000000     0.000000
6007.000000     0.002989
6009.000000     0.000000
6011.000000     0.021261
6013.000000     0.000290
6015.000000     0.000413
6017.000000     0.000318
6018.186441     0.406780
6019.000000     0.003126
6021.000000     0.000317
6023.000000     0.000000
6025.000000     0.000000
6027.000000     0.014376
6029.000000     0.028546
6031.000000     0.071114
6033.000000     0.001236
6035.000000     0.001203
6037.000000     0.001596
6039.000000     0.034914
6041.000000     0.000210
6043.000000     0.000800
6045.000000     0.011587
6047.000000     0.002166
6049.000000     0.000000
6051.000000     0.031347
6053.000000     0.002903
6055.000000     0.020527
6057.000000     0.002446
                  ...   
55083.000000    0.006026
55085.000000    0.000000
55087.000000    0.010251
55089.000000    0.000108
55091.000000    0.015381
55093.000000    0.000000
55095.000000    0.014491
55097.000000    0.000074
55099.000000    0.00

## Drought index by county

In [9]:
data_drought['valid_start'] = pd.to_datetime(data_drought['valid_start'])
data_drought['valid_end'] = pd.to_datetime(data_drought['valid_end'])

In [10]:
data_drought['event_duration_in_days'] = (data_drought['valid_end'] - data_drought['valid_start']).dt.days

In [11]:
data_drought['event_importance_with_weight'] = data_drought['event_duration_in_days'] * 0. + (data_drought['d0']*1. + data_drought['d1']*2. + data_drought['d2']*3. + data_drought['d3']*4. + data_drought['d4']*5.) / 100.
groups = data_drought.groupby("fips")
drought_index = groups['event_importance_with_weight'].sum() / groups['event_duration_in_days'].sum()

In [12]:
data_drought

,fips,county,state,none,d0,d1,d2,d3,d4,valid_start,valid_end,event_duration_in_days,event_importance_with_weight
0,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2017-12-26,2018-01-01,6,0.0000
1,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2017-12-19,2017-12-25,6,0.0000
2,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2017-12-12,2017-12-18,6,0.0000
3,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2017-12-05,2017-12-11,6,0.0000
4,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2017-11-28,2017-12-04,6,0.0000
5,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2017-11-21,2017-11-27,6,0.0000
6,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2017-11-14,2017-11-20,6,0.0000
7,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2017-11-07,2017-11-13,6,0.0000
8,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2017-10-31,2017-11-06,6,0.0000
9,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,2017-10-24,2017-10-30,6,0.0000


In [13]:
drought_index

fips
1001     0.145533
1003     0.143284
1005     0.234105
1007     0.119415
1009     0.109820
1011     0.206507
1013     0.151655
1015     0.138964
1017     0.272743
1019     0.125913
1021     0.137915
1023     0.087852
1025     0.112923
1027     0.223499
1029     0.173111
1031     0.191757
1033     0.089665
1035     0.163774
1037     0.194750
1039     0.190085
1041     0.178382
1043     0.094834
1045     0.213561
1047     0.103573
1049     0.109051
1051     0.213187
1053     0.166651
1055     0.106476
1057     0.096223
1059     0.080830
           ...   
72095    0.052976
72097    0.010433
72099    0.006119
72101    0.034560
72103    0.040788
72105    0.061053
72107    0.036815
72109    0.083343
72111    0.027675
72113    0.035986
72115    0.008658
72117    0.008761
72119    0.049147
72121    0.013492
72123    0.141241
72125    0.009403
72127    0.053814
72129    0.069353
72131    0.002986
72133    0.101331
72135    0.051546
72137    0.037743
72139    0.071403
72141    0.005447
72143

## Education index by county

In [14]:
data_education

,fips,state,county,year,less_than_hs,hs_diploma,some_college_or_associates,college_bachelors_or_higher,pct_less_than_hs,pct_hs_diploma,pct_college_or_associates,pct_college_bachelors_or_higher,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,0,US,United States,1970,52373312.0,34158051.0,11650730.0,11717266.0,47.7,31.1,10.6,10.7,NaN,NaN,NaN
1,1000,AL,Alabama,1970,1062306.0,468269.0,136287.0,141936.0,58.7,25.9,7.5,7.8,NaN,NaN,NaN
2,1001,AL,Autauga County,1970,6611.0,3757.0,933.0,767.0,54.8,31.1,7.7,6.4,NaN,NaN,NaN
3,1003,AL,Baldwin County,1970,18726.0,8426.0,2334.0,2038.0,59.4,26.7,7.4,6.5,NaN,NaN,NaN
4,1005,AL,Barbour County,1970,8120.0,2242.0,581.0,861.0,68.8,19.0,4.9,7.3,NaN,NaN,NaN
5,1007,AL,Bibb County,1970,5272.0,1402.0,238.0,302.0,73.1,19.4,3.3,4.2,NaN,NaN,NaN
6,1009,AL,Blount County,1970,10677.0,3440.0,626.0,404.0,70.5,22.7,4.1,2.7,NaN,NaN,NaN
7,1011,AL,Bullock County,1970,4245.0,958.0,305.0,314.0,72.9,16.5,5.2,5.4,NaN,NaN,NaN
8,1013,AL,Butler County,1970,8353.0,2459.0,499.0,541.0,70.5,20.7,4.2,4.6,NaN,NaN,NaN
9,1015,AL,Calhoun County,1970,30535.0,13804.0,3823.0,3921.0,58.6,26.5,7.3,7.5,NaN,NaN,NaN


In [15]:
data_education['avg_higher_education_years'] = (data_education['pct_less_than_hs'] * (-3) + data_education['pct_hs_diploma'] * 0. + data_education['pct_college_or_associates']*3. + data_education['pct_college_bachelors_or_higher']*4.)/100.

In [16]:
data_education

,fips,state,county,year,less_than_hs,hs_diploma,some_college_or_associates,college_bachelors_or_higher,pct_less_than_hs,pct_hs_diploma,pct_college_or_associates,pct_college_bachelors_or_higher,Unnamed: 12,Unnamed: 13,Unnamed: 14,avg_higher_education_years
0,0,US,United States,1970,52373312.0,34158051.0,11650730.0,11717266.0,47.7,31.1,10.6,10.7,NaN,NaN,NaN,-0.685
1,1000,AL,Alabama,1970,1062306.0,468269.0,136287.0,141936.0,58.7,25.9,7.5,7.8,NaN,NaN,NaN,-1.224
2,1001,AL,Autauga County,1970,6611.0,3757.0,933.0,767.0,54.8,31.1,7.7,6.4,NaN,NaN,NaN,-1.157
3,1003,AL,Baldwin County,1970,18726.0,8426.0,2334.0,2038.0,59.4,26.7,7.4,6.5,NaN,NaN,NaN,-1.300
4,1005,AL,Barbour County,1970,8120.0,2242.0,581.0,861.0,68.8,19.0,4.9,7.3,NaN,NaN,NaN,-1.625
5,1007,AL,Bibb County,1970,5272.0,1402.0,238.0,302.0,73.1,19.4,3.3,4.2,NaN,NaN,NaN,-1.926
6,1009,AL,Blount County,1970,10677.0,3440.0,626.0,404.0,70.5,22.7,4.1,2.7,NaN,NaN,NaN,-1.884
7,1011,AL,Bullock County,1970,4245.0,958.0,305.0,314.0,72.9,16.5,5.2,5.4,NaN,NaN,NaN,-1.815
8,1013,AL,Butler County,1970,8353.0,2459.0,499.0,541.0,70.5,20.7,4.2,4.6,NaN,NaN,NaN,-1.805
9,1015,AL,Calhoun County,1970,30535.0,13804.0,3823.0,3921.0,58.6,26.5,7.3,7.5,NaN,NaN,NaN,-1.239


### Water usage

In [21]:
data_water

,state,state_fips,county,county_fips,fips,year,population,pub_sup_1,pub_sup_2,pub_sup_3,...,thermoelectric_30,gro_wat_1,gro_wat_2,gro_wat_3,surf_wat_1,surf_wat_2,surf_wat_3,total_withdrawal_1,total_withdrawal_2,total_withdrawal_3
0,AL,1,Autauga County,1,1001,2010,54.571,NaN,NaN,48.222,...,6738.26,18.38,0.0,18.38,37.17,0.00,37.17,55.55,0.00,55.55
1,AL,1,Baldwin County,3,1003,2010,182.265,NaN,NaN,153.463,...,0.00,57.35,0.0,57.35,7.01,0.00,7.01,64.36,0.00,64.36
2,AL,1,Barbour County,5,1005,2010,27.457,NaN,NaN,25.555,...,0.00,6.71,0.0,6.71,4.34,0.00,4.34,11.05,0.00,11.05
3,AL,1,Bibb County,7,1007,2010,22.915,NaN,NaN,21.279,...,0.00,6.06,0.0,6.06,1.55,0.00,1.55,7.61,0.00,7.61
4,AL,1,Blount County,9,1009,2010,57.322,NaN,NaN,44.464,...,0.00,4.04,0.0,4.04,53.04,0.00,53.04,57.08,0.00,57.08
5,AL,1,Bullock County,11,1011,2010,10.914,NaN,NaN,10.176,...,0.00,2.76,0.0,2.76,0.43,0.00,0.43,3.19,0.00,3.19
6,AL,1,Butler County,13,1013,2010,20.947,NaN,NaN,17.599,...,0.00,3.73,0.0,3.73,0.79,0.00,0.79,4.52,0.00,4.52
7,AL,1,Calhoun County,15,1015,2010,118.572,NaN,NaN,112.390,...,0.00,22.51,0.0,22.51,6.80,0.00,6.80,29.31,0.00,29.31
8,AL,1,Chambers County,17,1017,2010,34.215,NaN,NaN,25.875,...,0.00,0.95,0.0,0.95,4.45,0.00,4.45,5.40,0.00,5.40
9,AL,1,Cherokee County,19,1019,2010,25.989,NaN,NaN,17.876,...,0.00,3.17,0.0,3.17,2.37,0.00,2.37,5.54,0.00,5.54


In [19]:
water_info

,state,State postal abbreviation
0,state_fips,State FIPS code
1,county,County name
2,county_fips,County FIPS code
3,fips,Concatenated State-county FIPS code
4,year,Year of data=2010
5,population,"Total population of county, in thousands"
6,pub_sup_1,"Public Supply, population served by groundwate..."
7,pub_sup_2,"Public Supply, population served by surface wa..."
8,pub_sup_3,"Public Supply, total population served, in tho..."
9,pub_sup_4,"Public Supply, groundwater withdrawals, fresh..."


In [47]:
new_cat = ['Domestic', 'Public', 'Irrigation_acres', 'Irrigation_crop', 'Irrigation_golf', 'Mining', 'Industry', 'Livestock',
           'Aquaculture', 'Thermoelectric', 'Ground', 'Surface', 'Total']
identifiers = ['dom', 'pub', 'irr', 'cro', 'gol', 'min', 'in', 'live',
           'aqua', 'thermo', 'gro', 'surf', 'total']

for i in range(13):
    data_water[new_cat[i]] = data_water.loc[:,list(data_water.columns[data_water.columns.str.startswith(identifiers[i])])].sum(axis=1)


data_water.loc[:,new_cat]

,Domestic,Public,Irrigation_acres,Irrigation_crop,Irrigation_golf,Mining,Industry,Livestock,Aquaculture,Thermoelectric,Ground,Surface,Total
0,275.598,68.582,9.78,9.46,0.32,0.52,164.12,0.30,0.00,13523.24,36.76,74.34,111.10
1,389.664,245.343,104.00,96.54,7.46,0.84,0.00,0.70,0.64,0.00,114.70,14.02,128.72
2,318.304,42.155,10.94,9.96,0.98,0.76,6.32,0.76,9.72,0.00,13.42,8.68,22.10
3,352.932,40.839,2.18,1.86,0.32,1.44,0.00,0.14,5.80,0.00,12.12,3.10,15.22
4,314.976,262.904,2.90,1.60,1.30,0.48,0.00,1.96,0.00,0.00,8.08,106.08,114.16
5,442.436,19.376,1.70,1.52,0.18,0.24,0.00,0.28,0.00,0.00,5.52,0.86,6.38
6,333.736,28.399,1.56,0.18,1.38,0.00,1.20,0.80,2.00,0.00,7.46,1.58,9.04
7,385.024,205.590,14.88,11.82,3.06,0.40,3.88,0.64,0.08,0.00,45.02,13.60,58.62
8,363.380,43.115,1.18,1.00,0.18,0.00,0.00,0.36,0.00,0.00,1.90,8.90,10.80
9,279.946,31.836,5.38,5.06,0.32,0.04,0.00,0.56,0.00,0.00,6.34,4.74,11.08


In [ ]:
groups = data_water.groupby("fips","year")[['greater_than_MCL', 'fips', "pop_served"]].mean()
groups['index_with_weights'] = groups['greater_than_MCL']*groups['pop_served']
chemicals_index = groups.groupby("fips")['index_with_weights'].sum() / groups.groupby("fips")['pop_served'].sum()